<a href="https://colab.research.google.com/github/CoolerKula/Introduction-To-Computer-Vision/blob/main/Introduction_To_Computer_Vision_project_Bernard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Description**

The supermarket chain would like to explore whether Data Science can help them adhere to alcohol laws by making sure they do not sell alcohol to people underage. You are asked to conduct that evaluation, so as you set to work, keep the following in mind:

● The shops are equipped with cameras in the checkout area which are triggered when a person is buying alcohol.

● Computer vision methods can be used to determine the age of a person from a photo.

● The task then is to build and evaluate a model for verifying people's age.
To start working on the task, you'll have a set of photographs of people with their ages indicated.

**Project Instructions**

● Before getting your hands dirty with the project, let's think of the Good Seed supermarket task in detail. Imagine your manager wants to check your understanding of the task.
Pass this quiz to verify your understanding of the project statement.

● Perform exploratory data analysis to get an overall impression of the dataset.

● Train and evaluate the model (it needs to be done on the GPU platform).

● Combine your code, output, and findings (from the previous points) in the final notebook.

● Make conclusions of the model evaluation, add them to the notebook.
Data description

Download the dataset for this project here [link].

As the number of image files is rather high, it is advisable to avoid reading them all at once, which would be resource-consuming, and to read them sequentially with the ImageDataGenerator generator. This method was explained in Data Generators.